In [1]:
import org.apache.spark.sql.SparkSession

In [2]:
val spark = SparkSession
  .builder()
  .appName("lab5_scala")
  .config("spark.executor.instances", "3")
  .getOrCreate()

spark = org.apache.spark.sql.SparkSession@4f429c56


org.apache.spark.sql.SparkSession@4f429c56

In [3]:
import org.apache.spark.sql.types._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark._

In [4]:
import spark.implicits._

In [5]:
val train_data = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("lab05_train.csv").repartition(9)
val test_data = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("lab05_test.csv").repartition(9)

train_data = [_c0: int, ID: int ... 115 more fields]
test_data = [_c0: int, ID: int ... 114 more fields]


[_c0: int, ID: int ... 114 more fields]

In [6]:
train_data.printSchema

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- CR_PROD_CNT_IL: integer (nullable = true)
 |-- AMOUNT_RUB_CLO_PRC: double (nullable = true)
 |-- PRC_ACCEPTS_A_EMAIL_LINK: double (nullable = true)
 |-- APP_REGISTR_RGN_CODE: double (nullable = true)
 |-- PRC_ACCEPTS_A_POS: double (nullable = true)
 |-- PRC_ACCEPTS_A_TK: double (nullable = true)
 |-- TURNOVER_DYNAMIC_IL_1M: double (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY1M: double (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY1M: double (nullable = true)
 |-- AMOUNT_RUB_SUP_PRC: double (nullable = true)
 |-- PRC_ACCEPTS_A_AMOBILE: double (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY3M: double (nullable = true)
 |-- CLNT_TRUST_RELATION: string (nullable = true)
 |-- PRC_ACCEPTS_TK: double (nullable = true)
 |-- PRC_ACCEPTS_A_MTP: double (nullable = true)
 |-- REST_DYNAMIC_FDEP_1M: double (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY3M: double (nullable = true)
 |-- CNT_ACCEPTS_TK: double (nullable = true)
 

In [7]:
val null_thresh = 320764*0.5               

val to_drop = train_data.columns.filter(
    c => train_data.agg(
        sum(when(train_data(c).isNull, 1).otherwise(0)).alias(c)
    ).first().getLong(0) > null_thresh
)

null_thresh = 160382.0
to_drop = Array(PRC_ACCEPTS_A_EMAIL_LINK, APP_REGISTR_RGN_CODE, PRC_ACCEPTS_A_POS, PRC_ACCEPTS_A_TK, CNT_TRAN_AUT_TENDENCY1M, SUM_TRAN_AUT_TENDENCY1M, PRC_ACCEPTS_A_AMOBILE, SUM_TRAN_AUT_TENDENCY3M, CLNT_TRUST_RELATION, PRC_ACCEPTS_TK, PRC_ACCEPTS_A_MTP, CNT_TRAN_AUT_TENDENCY3M, CNT_ACCEPTS_TK, APP_MARITAL_STATUS, CNT_TRAN_MED_TENDENCY1M, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, APP_DRIVING_LICENSE, APP_EDUCATION, CNT_TRAN_CLO_TENDENCY1M, SUM_TRAN_MED_TENDENCY1M, PRC_ACCEPTS_A_ATM, PRC_ACCEPTS_MTP, APP_TRAVEL_PASS, CNT_ACCEPTS_MTP, APP_CAR, SUM_TRAN_CLO_TENDENCY1M, APP_POSITION_TYPE, CNT_TRAN_MED_TENDENCY3M, SUM_TRAN_MED_TENDENCY3M, APP_EMP_TYPE, SUM_TRAN_CLO_TENDENCY3M, LDEAL_TENOR_MAX, LDEAL_YQZ_CHRG, DEAL_YQZ_IR_MAX, LDEAL_YQZ...


Array(PRC_ACCEPTS_A_EMAIL_LINK, APP_REGISTR_RGN_CODE, PRC_ACCEPTS_A_POS, PRC_ACCEPTS_A_TK, CNT_TRAN_AUT_TENDENCY1M, SUM_TRAN_AUT_TENDENCY1M, PRC_ACCEPTS_A_AMOBILE, SUM_TRAN_AUT_TENDENCY3M, CLNT_TRUST_RELATION, PRC_ACCEPTS_TK, PRC_ACCEPTS_A_MTP, CNT_TRAN_AUT_TENDENCY3M, CNT_ACCEPTS_TK, APP_MARITAL_STATUS, CNT_TRAN_MED_TENDENCY1M, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, APP_DRIVING_LICENSE, APP_EDUCATION, CNT_TRAN_CLO_TENDENCY1M, SUM_TRAN_MED_TENDENCY1M, PRC_ACCEPTS_A_ATM, PRC_ACCEPTS_MTP, APP_TRAVEL_PASS, CNT_ACCEPTS_MTP, APP_CAR, SUM_TRAN_CLO_TENDENCY1M, APP_POSITION_TYPE, CNT_TRAN_MED_TENDENCY3M, SUM_TRAN_MED_TENDENCY3M, APP_EMP_TYPE, SUM_TRAN_CLO_TENDENCY3M, LDEAL_TENOR_MAX, LDEAL_YQZ_CHRG, DEAL_YQZ_IR_MAX, LDEAL_YQZ...

In [15]:
val train_cleaned = train_data.drop(to_drop : _*).drop("_c0").where(col("TARGET").isNotNull)
val test_cleaned = test_data.drop(to_drop : _*).drop("_c0")

train_cleaned = [ID: int, CR_PROD_CNT_IL: int ... 49 more fields]
test_cleaned = [ID: int, CR_PROD_CNT_IL: int ... 48 more fields]


[ID: int, CR_PROD_CNT_IL: int ... 48 more fields]

In [20]:
val numeric_columns = train_cleaned.schema.fields.filter(x => x.dataType == DoubleType).map(x => x.name)

numeric_columns = Array(AMOUNT_RUB_CLO_PRC, TURNOVER_DYNAMIC_IL_1M, AMOUNT_RUB_SUP_PRC, REST_DYNAMIC_FDEP_1M, REST_DYNAMIC_SAVE_3M, REST_AVG_CUR, AMOUNT_RUB_NAS_PRC, TRANS_COUNT_SUP_PRC, TRANS_COUNT_NAS_PRC, TURNOVER_CC, TRANS_COUNT_ATM_PRC, AMOUNT_RUB_ATM_PRC, TURNOVER_PAYM, REST_DYNAMIC_FDEP_3M, REST_DYNAMIC_IL_1M, REST_DYNAMIC_CUR_1M, REST_AVG_PAYM, LDEAL_GRACE_DAYS_PCT_MED, REST_DYNAMIC_CUR_3M, CNT_TRAN_SUP_TENDENCY3M, TURNOVER_DYNAMIC_CUR_1M, REST_DYNAMIC_PAYM_3M, SUM_TRAN_SUP_TENDENCY3M, REST_DYNAMIC_IL_3M, CNT_TRAN_ATM_TENDENCY3M, CNT_TRAN_ATM_TENDENCY1M, TURNOVER_DYNAMIC_IL_3M, SUM_TRAN_ATM_TENDENCY3M, SUM_TRAN_ATM_TENDENCY1M, REST_DYNAMIC_PAYM_1M, TURNOVER_DYNAMIC_CUR_3M, CLNT_SETUP_TENOR, TURNOVER_DYNAMIC_PAYM_3M, TURNOVER_DYNAMIC_PAYM_1M, TRANS_AMOUNT_TENDENCY3...


Array(AMOUNT_RUB_CLO_PRC, TURNOVER_DYNAMIC_IL_1M, AMOUNT_RUB_SUP_PRC, REST_DYNAMIC_FDEP_1M, REST_DYNAMIC_SAVE_3M, REST_AVG_CUR, AMOUNT_RUB_NAS_PRC, TRANS_COUNT_SUP_PRC, TRANS_COUNT_NAS_PRC, TURNOVER_CC, TRANS_COUNT_ATM_PRC, AMOUNT_RUB_ATM_PRC, TURNOVER_PAYM, REST_DYNAMIC_FDEP_3M, REST_DYNAMIC_IL_1M, REST_DYNAMIC_CUR_1M, REST_AVG_PAYM, LDEAL_GRACE_DAYS_PCT_MED, REST_DYNAMIC_CUR_3M, CNT_TRAN_SUP_TENDENCY3M, TURNOVER_DYNAMIC_CUR_1M, REST_DYNAMIC_PAYM_3M, SUM_TRAN_SUP_TENDENCY3M, REST_DYNAMIC_IL_3M, CNT_TRAN_ATM_TENDENCY3M, CNT_TRAN_ATM_TENDENCY1M, TURNOVER_DYNAMIC_IL_3M, SUM_TRAN_ATM_TENDENCY3M, SUM_TRAN_ATM_TENDENCY1M, REST_DYNAMIC_PAYM_1M, TURNOVER_DYNAMIC_CUR_3M, CLNT_SETUP_TENOR, TURNOVER_DYNAMIC_PAYM_3M, TURNOVER_DYNAMIC_PAYM_1M, TRANS_AMOUNT_TENDENCY3...

In [21]:
val imputer_output = numeric_columns.map(x => x.concat("_imputed"))

imputer_output = Array(AMOUNT_RUB_CLO_PRC_imputed, TURNOVER_DYNAMIC_IL_1M_imputed, AMOUNT_RUB_SUP_PRC_imputed, REST_DYNAMIC_FDEP_1M_imputed, REST_DYNAMIC_SAVE_3M_imputed, REST_AVG_CUR_imputed, AMOUNT_RUB_NAS_PRC_imputed, TRANS_COUNT_SUP_PRC_imputed, TRANS_COUNT_NAS_PRC_imputed, TURNOVER_CC_imputed, TRANS_COUNT_ATM_PRC_imputed, AMOUNT_RUB_ATM_PRC_imputed, TURNOVER_PAYM_imputed, REST_DYNAMIC_FDEP_3M_imputed, REST_DYNAMIC_IL_1M_imputed, REST_DYNAMIC_CUR_1M_imputed, REST_AVG_PAYM_imputed, LDEAL_GRACE_DAYS_PCT_MED_imputed, REST_DYNAMIC_CUR_3M_imputed, CNT_TRAN_SUP_TENDENCY3M_imputed, TURNOVER_DYNAMIC_CUR_1M_imputed, REST_DYNAMIC_PAYM_3M_imputed, SUM_TRAN_SUP_TENDENCY3M_imputed, REST_DYNAMIC_IL_3M_imputed, CNT_TRAN_ATM_TENDENCY3M_imputed, CNT_TRAN_ATM_TENDENCY1M_imputed, TURNOV...


Array(AMOUNT_RUB_CLO_PRC_imputed, TURNOVER_DYNAMIC_IL_1M_imputed, AMOUNT_RUB_SUP_PRC_imputed, REST_DYNAMIC_FDEP_1M_imputed, REST_DYNAMIC_SAVE_3M_imputed, REST_AVG_CUR_imputed, AMOUNT_RUB_NAS_PRC_imputed, TRANS_COUNT_SUP_PRC_imputed, TRANS_COUNT_NAS_PRC_imputed, TURNOVER_CC_imputed, TRANS_COUNT_ATM_PRC_imputed, AMOUNT_RUB_ATM_PRC_imputed, TURNOVER_PAYM_imputed, REST_DYNAMIC_FDEP_3M_imputed, REST_DYNAMIC_IL_1M_imputed, REST_DYNAMIC_CUR_1M_imputed, REST_AVG_PAYM_imputed, LDEAL_GRACE_DAYS_PCT_MED_imputed, REST_DYNAMIC_CUR_3M_imputed, CNT_TRAN_SUP_TENDENCY3M_imputed, TURNOVER_DYNAMIC_CUR_1M_imputed, REST_DYNAMIC_PAYM_3M_imputed, SUM_TRAN_SUP_TENDENCY3M_imputed, REST_DYNAMIC_IL_3M_imputed, CNT_TRAN_ATM_TENDENCY3M_imputed, CNT_TRAN_ATM_TENDENCY1M_imputed, TURNOV...

In [24]:
import org.apache.spark.ml.feature.{Imputer, VectorAssembler}
import org.apache.spark.ml.Pipeline

In [27]:
val imputer = new Imputer().setInputCols(numeric_columns).setOutputCols(imputer_output)
val assembler = new VectorAssembler().setInputCols(imputer.getOutputCols).setOutputCol("features")
val pipeline = new Pipeline().setStages(Array(imputer, assembler))
val fitted_pipeline = pipeline.fit(train_cleaned)

imputer = imputer_3a86f38b38e3
assembler = vecAssembler_15502a61d80c
pipeline = pipeline_274d07bef531
fitted_pipeline = pipeline_274d07bef531


pipeline_274d07bef531

In [28]:
val train_features = fitted_pipeline.transform(train_cleaned).select("ID", "features", "TARGET")
val test_features = fitted_pipeline.transform(test_cleaned).select("ID", "features")

train_features = [ID: int, features: vector ... 1 more field]
test_features = [ID: int, features: vector]


[ID: int, features: vector]

In [29]:
import org.apache.spark.ml.regression.GBTRegressor

In [30]:
val gbt = new GBTRegressor()
  .setLabelCol("TARGET")
  .setFeaturesCol("features")
  .setMaxIter(10)

gbt = gbtr_ad73429638ed


gbtr_ad73429638ed

In [31]:
val gbtModel = gbt.fit(train_features)

gbtModel = GBTRegressionModel (uid=gbtr_ad73429638ed) with 10 trees


GBTRegressionModel (uid=gbtr_ad73429638ed) with 10 trees

In [32]:
val predictions = gbtModel.transform(test_features)

predictions = [ID: int, features: vector ... 1 more field]


[ID: int, features: vector ... 1 more field]

In [33]:
predictions.printSchema

root
 |-- ID: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [35]:
val formatted_predictions = predictions.select(col("ID").alias("id"), col("prediction").alias("target"))

formatted_predictions = [id: int, target: double]


[id: int, target: double]

In [36]:
formatted_predictions.show

+------+--------------------+
|    id|              target|
+------+--------------------+
|412526|   0.062531579825321|
|287795| 0.05801986839747308|
|168477| 0.14907131236934534|
|164955| 0.08395443833268922|
|506312|0.026725001717987983|
|168433| 0.08012326979662696|
|432893| 0.00418002077730148|
|558248|   0.026419577050117|
|303459|0.013296422312620557|
|369168| 0.23351626196083528|
|408332| 0.15204690822343314|
|433082| 0.11494041973074494|
|422373| 0.08809626087407926|
|513560| 0.41764320315357634|
|299674|0.002980147483337211|
|523888|0.004077939595414096|
|444091|-0.00153809600934...|
|262653| 0.04103512650894509|
|553431| 0.09115685947485448|
|484155| 0.07388045294636353|
+------+--------------------+
only showing top 20 rows



In [37]:
formatted_predictions.coalesce(1).write.mode(SaveMode.Overwrite).option("delimiter", "\t").option("header", "true").csv("lab05.csv")

In [38]:
sc.stop